In [1]:
import pandas as pd

In [6]:
data = [[1, 'Joe', 70000, 1], [2, 'Jim', 90000, 1], [3, 'Henry', 80000, 2], [4, 'Sam', 60000, 2], [5, 'Max', 90000, 1]]
employee = pd.DataFrame(data, columns=['id', 'name', 'salary', 'departmentId']).astype({'id':'Int64', 'name':'object', 'salary':'Int64', 'departmentId':'Int64'})
data = [[1, 'IT'], [2, 'Sales']]
department = pd.DataFrame(data, columns=['id', 'name']).astype({'id':'Int64', 'name':'object'})
department
employee

,id,name,salary,departmentId
0,1,Joe,70000,1
1,2,Jim,90000,1
2,3,Henry,80000,2
3,4,Sam,60000,2
4,5,Max,90000,1


In [11]:
df = pd.merge(employee, department, left_on='departmentId', right_on='id', how='left', suffixes=('_emp', '_dep'))
df

,id_emp,name_emp,salary,departmentId,id_dep,name_dep
0,1,Joe,70000,1,1,IT
1,2,Jim,90000,1,1,IT
2,3,Henry,80000,2,2,Sales
3,4,Sam,60000,2,2,Sales
4,5,Max,90000,1,1,IT


In [25]:
max_salary = df.groupby('departmentId')['salary'].transform('max') == df['salary']
dt = df[max_salary].rename(columns={'name_dep': 'Department', 'name_emp': 'Employee', 'salary': 'Salary'})[['Department', 'Employee', 'Salary']]
dt

,Department,Employee,Salary
1,IT,Jim,90000
2,Sales,Henry,80000
4,IT,Max,90000


In [45]:
data = [[0, 95, 100, 105], [1, 70, None, 80]]
products = pd.DataFrame(data, columns=['product_id', 'store1', 'store2', 'store3']).astype({'product_id':'Int64', 'store1':'Int64', 'store2':'Int64', 'store3':'Int64'})
products

,product_id,store1,store2,store3
0,0,95,100,105
1,1,70,<NA>,80


In [49]:
df = products.set_index('product_id').stack(dropna=True).reset_index()
df

,product_id,level_1,0
0,0,store1,95
1,0,store2,100
2,0,store3,105
3,1,store1,70
4,1,store3,80


In [ ]:
products.columns = ["product_id", "store", "price"]

In [ ]:
products.sort_values(["product_id", "store"])


In [92]:
data = [[1, 1, 'Clicked'], [2, 2, 'Clicked'], [3, 3, 'Viewed'], [5, 5, 'Ignored'], [1, 7, 'Ignored'], [2, 7, 'Viewed'], [3, 5, 'Clicked'], [1, 4, 'Viewed'], [2, 11, 'Viewed'], [1, 2, 'Clicked']]
ads = pd.DataFrame(data, columns=['ad_id', 'user_id', 'action']).astype({'ad_id':'Int64', 'user_id':'Int64', 'action':'object'})
ads

,ad_id,user_id,action
0,1,1,Clicked
1,2,2,Clicked
2,3,3,Viewed
3,5,5,Ignored
4,1,7,Ignored
5,2,7,Viewed
6,3,5,Clicked
7,1,4,Viewed
8,2,11,Viewed
9,1,2,Clicked


In [106]:
# 计算这个点击次数和总次数
clicks = ads[ads['action']=='Clicked'].groupby('ad_id')['action'].count()
# valid_totals 过滤，如果是Ignored则为0，并统计非拒绝的数据
valid_totals = ads.apply(lambda row:0 if row['action']=='Ignored' else 1,axis=1).groupby(ads['ad_id']).sum()
print(valid_totals)
print(clicks)
str = ((clicks/valid_totals)*100).fillna(0).round(2)
print(str)
str.reset_index(name='ctr')

ad_id
1    3
2    3
3    2
5    0
dtype: int64
ad_id
1    2
2    1
3    1
Name: action, dtype: int64
ad_id
1    66.67
2    33.33
3    50.00
5     0.00
dtype: float64


,ad_id,ctr
0,1,66.67
1,2,33.33
2,3,50.00
3,5,0.00


In [94]:
import pandas as pd

def ads_performance(ads: pd.DataFrame) -> pd.DataFrame:
   
    # 计算这个点击次数和总次数
    clicks = ads[ads['action']=='Clicked'].groupby('ad_id')['action'].count()
    
    # valid_totals 过滤，如果是Ignored则为0，并统计非拒绝的数据
    valid_totals = ads.apply(lambda row:0 if row['action']=='Ignored' else 1,axis=1).groupby(ads['ad_id']).sum()
   

    # 计算CTR值
    ctr = (clicks/valid_totals)*100
    ctr = ctr.round(2)
    # 将缺失值填充为0
    ctr = ctr.fillna(0)
    

    # 返回结果按照ctr降序、ad_id 升序进行排序
    ans = ctr.reset_index(name='ctr').sort_values(by=['ctr','ad_id'],ascending=[False,True])

    return ans
ads_performance(ads)


,ad_id,ctr
0,1,66.67
2,3,50.00
1,2,33.33
3,5,0.00


In [83]:
df = ads.pivot_table(index='ad_id', columns='action', values='user_id', aggfunc='count', fill_value=0).reset_index().rename_axis(None, axis=1)
df

,ad_id,Clicked,Ignored,Viewed
0,1,2,1,1
1,2,1,0,2
2,3,1,0,1
3,5,0,1,0


In [91]:
df['ctr'] = ((df['Clicked'] / (df['Clicked'] + df['Viewed'])) * 100).round(2)
df['ctr'](0, inplace=True)
df

,ad_id,Clicked,Ignored,Viewed,ctr
0,1,2,1,1,66.67
1,2,1,0,2,33.33
2,3,1,0,1,50.0
3,5,0,1,0,NaN


In [71]:
import pandas as pd
import numpy as np

# 创建一个 DataFrame
df = pd.DataFrame({
    'A': ['foo', 'foo', 'foo', 'bar', 'bar', 'bar'],
    'B': ['one', 'one', 'two', 'two', 'one', 'one'],
    'C': ['small', 'large', 'large', 'small', 'small', 'large'],
    'D': [1, 2, 2, 3, 3, 4],
    'E': [2, 4, 5, 5, 6, 6]
})

# 创建一个透视表
pivot_table = df.pivot_table(values='D', index=['A', 'B'], columns=['C'], aggfunc='sum').reset_index().rename_axis(None, axis=1)

print(pivot_table)
df

     A    B  large  small
0  bar  one    4.0    3.0
1  bar  two    NaN    3.0
2  foo  one    2.0    1.0
3  foo  two    2.0    NaN


,A,B,C,D,E
0,foo,one,small,1,2
1,foo,one,large,2,4
2,foo,two,large,2,5
3,bar,two,small,3,5
4,bar,one,small,3,6
5,bar,one,large,4,6
